In [ ]:
pip install ortools gmplot folium polyline

     |████████████████████████████████| 14.4MB 269kB/s 
     |████████████████████████████████| 174kB 52.8MB/s 
     |████████████████████████████████| 1.0MB 41.6MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
def header(text):
    raw_html = '<h1>' + str(text) + '</h1>'
    return raw_html

def box(text):
    raw_html = '<div style="border:1px dotted black;padding:2em;">'+str(text)+'</div>'
    return raw_html

def showHTML(raw_html):
    htmlContent=''
    htmlContent += raw_html
    return HTML(htmlContent)

# Example
#showHTML( header("This is a header") )
# showHTML( box("This is some text in a box") )


# **Import Libraries**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import math
# import time 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from requests.structures import CaseInsensitiveDict
from functools import partial
from IPython.display import display, HTML
import random
import time
import datetime as dt
from pprint import pprint
print('Done!')

Done!


# **Importing Files**

In [ ]:
ps = pd.ExcelFile(r'/content/drive/MyDrive/The Kabadiwala/VRP Data/March_PickupSession_data.xlsx')
pickup = pd.read_excel(ps, 'Sheet1') # pickup data
mat =  pd.read_excel(ps, 'Sheet2') # material data

In [ ]:
pickup['HandleTime'] = pd.to_datetime(pickup['HandleTime'], utc=True) # Fixing datatype of column.
# Spliting DateTIme to separate columns
pickup['HandleDate'] = pickup['HandleTime'].dt.date
pickup['HandleTime'] = pickup['HandleTime'].dt.time

# Selecting Request Handling Time by DriverApp, as it is most accurate than others. 
_pickup = pickup[['RequestId', 'PickupSessionId', 'AddressTypeId', 'AddressTypeName','FranchiseId','AppLatitude', 'AppLongitude', 'HandleTime', 'HandleDate']]
_pickup.rename(columns= {'AppLatitude':'Latitude','AppLongitude':'Longitude'},inplace = True)

In [ ]:
# Selecting a specific date to operate
_date = '19/03/2021'
# Base Franchises
Franchises ={'Indore':2,'Bhopal_1':4,'Bhopal_2':6,'Raipur':8,'Nagpur':9,"Lucknow":10}
# Franchise to Operate
selected_franchise =Franchises['Bhopal_1']

In [ ]:
pickup_all = _pickup[_pickup.HandleDate == dt.datetime.strptime(_date, '%d/%m/%Y').date()]
display(HTML(pickup_all[['RequestId', 'FranchiseId']].groupby(by =['FranchiseId']).count().sort_values(by='RequestId',ascending = False).rename(columns={'RequestId':'Request Count'}).to_html()))

,Request Count
FranchiseId,
4,20
2,15
8,5
10,5


In [ ]:
def add_weight_piece(pickup,material):
  # Adding Weight and Piece Column in _Pickup
  mat_sum = material.groupby(by = ['RequestId','Unit']).sum().reset_index()
  for index, row in pickup.iterrows():
    temp_df  = mat_sum[mat_sum.RequestId == row.RequestId]
    if temp_df[temp_df.Unit == 'Kg'].empty:
      kg =0
    else:
      kg = temp_df[temp_df.Unit == 'Kg'].Quantity.values[0]
    if temp_df[temp_df.Unit == 'Piece'].empty:
      piece =0
    else:
      piece = temp_df[temp_df.Unit == 'Piece'].Quantity.values[0]

    try:
      pickup.at[index,'weight'] = kg
      pickup.at[index,'piece'] = piece
    except Exception as e:
      print(f'Exception at {row.RequestId} :',e)
  return pickup

In [ ]:
Pickup = _pickup[(_pickup.HandleDate == dt.datetime.strptime(_date, '%d/%m/%Y').date())&(_pickup.FranchiseId == selected_franchise)]
_Pickup = add_weight_piece(Pickup,mat)
_Pickup = _Pickup.sort_values(by=['PickupSessionId','HandleTime']).reset_index(drop = True)
to_print = f'Pickup data initialized for date: {_date} and FranchiseId: {selected_franchise}'
showHTML(header(to_print))
HTML(_Pickup.head(5).to_html())
showHTML(box('Data Shape:'+' ' +str(_Pickup.shape)))

,RequestId,PickupSessionId,AddressTypeId,AddressTypeName,FranchiseId,Latitude,Longitude,HandleTime,HandleDate,weight,piece
0,76754,2892,7,Other,4,23.262548,77.450542,16:39:44.519000,2021-03-19,306.70,315.0
1,76782,2899,1,Home,4,23.217540,77.448834,11:09:35.638000,2021-03-19,60.89,4.0
2,76679,2899,2,Shop,4,23.220901,77.478622,11:26:50.277000,2021-03-19,18.90,0.0
3,76795,2899,1,Home,4,23.194841,77.463923,11:59:31.336000,2021-03-19,44.45,2.0
4,76812,2899,1,Home,4,23.192349,77.465202,12:04:20.124000,2021-03-19,9.00,0.0


# **Data Initialization**

In [ ]:
"""Stores the data for the problem"""
def open_data(vehicle_no=4,df=None,depot =1):
    global _df
    global error_open_data

    error_open_data = None

    _df = add_depots_df(depot,df)
    data = {}
    ######################################
    '''Distance Matrix'''
  
    Sample_x = _df[:]
    str_x =''
    for x, y in zip(Sample_x['Latitude'], Sample_x['Longitude']):
        str_x+=f'{y},{x};'
    str_x = str_x[:-1]

    url_x = f"http://router.project-osrm.org/table/v1/driving/{str_x}?annotations=distance"
    resp = requests.post(url_x)
    data_x = json.loads(resp.text)
    # print('response of data',data_x)
    data['distance_matrix'] =  data_x['distances']
    #########################################
    '''Time Matrix'''


    url_y = f"http://router.project-osrm.org/table/v1/driving/{str_x}"      
    resp = requests.post(url_y)
    data_y = json.loads(resp.text)
    lol = data_y['durations']

    for i in range(len(lol)):
        for j in range(len(lol[i])):
            lol[i][j]=math.ceil(lol[i][j]/60) #converting seconds to minutes
    data['time_matrix'] = lol[:]
    data['depot_no'] = depot
    #########################################
    '''Demand'''
    '''Demand is a list of Weight to be loaded at visits. '''

    _capacity = 750 # Capacity of a vehicle

    depot_demand = [-_capacity for x in range(depot-1)]
    depot_demand = [0] + depot_demand
    # visit_demand = [ random.randint(3,8) for x in range(len(df))]
    try:
      visit_demand =_df[_df.AddressTypeId != 0].weight.values # Here Demand is already known and extracted from pickup data.
    except:
      print('Exception in Demand!')
      visit_demand = [ 1 for x in range(len(df))]
    # Merging Demands of Depot and Visits
    res_list = [y for x in [depot_demand, visit_demand] for y in x]
    data['demands'] =  [int(x) for x in res_list]
    #########################################
    # Multiple Vehicle Capacitites
    
    data['vehicle_capacities'] = [750 for _ in range(vehicle_no)] # Provide list of Capacities and must equal to number of Vehicles.
     
    #########################################
    data['time_per_demand_unit'] = 0.5     # Time in minute required to load per unit of load(ie. loading time minutes per kg of load)
    data['num_vehicles'] = vehicle_no
    data['num_locations'] = len(data['distance_matrix'])
    data['vehicle_max_distance'] =150_000 #Maximum Travel Distance in meters
    data['vehicle_capacity'] = _capacity
    
    ##################
    data['vehicle_max_time'] = 480  #Time in Minutes 
    data['depot'] = 0
    data['time_windows'] = random_time_window(data['num_locations'],0,data['vehicle_max_time'],30,depot) # Generating Random Time Windows 

    return data


def random_time_window(number,start_time,end_time,time_interval,depot =1):
  temp_list = [(0,end_time) for _ in range(depot) ]
  temp_list[0]= (0,0)
  for x in range(number-len(temp_list)):
      i = random.randrange(start_time,end_time,time_interval)
      # temp_list.append((i,i+time_interval))
      ##############################
      #predefined Time window for all
      #(0,vehicle max time)
      temp_list.append((0,end_time))
      ##############################
  return temp_list


##############################################################################
# Check Below fuction as you update any the columns in _Pickup.
##############################################################################
# adding depots in DataFrame
def add_depots_df(number,data):
  depots = pd.DataFrame()
  columns = list(data)
  # print(data.FranchiseId)
  coordinates = find_franchises(int(data.FranchiseId.mode().values[0]))
  for i in range(number):
    '''
    COLUMNS: 
    ['RequestId', 'PickupSessionId', 'AddressTypeId', 'AddressTypeName','FranchiseId', 'Latitude', 'Longitude', 'HandleTime', 'HandleDate','weight', 'piece']
    '''
  
    values = [0,0,0,0,0,coordinates[0],coordinates[1],0,0,0,0]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
    #print(a_dictionary)
    depots = depots.append(a_dictionary,ignore_index= True)
  return  pd.concat([depots, data])
def find_franchises(FranchiseId):
  #print('in find franchises with frnachiseid',FranchiseId)
  if FranchiseId ==4 or FranchiseId ==6 :
    coordinates=[23.2637424,77.4547725]
  elif FranchiseId ==2:
    coordinates=[22.683603,75.9104723]
  elif FranchiseId ==8:
    coordinates=[21.260239,81.6216235]
  elif FranchiseId ==10:
    coordinates=[26.8901769,81.0017074]
  else:
    coordinates = None
  return coordinates

'Stores the data for the problem'

In [ ]:
{'Indore': 2, 'Bhopal_1': 4, 'Bhopal_2': 6, 'Raipur': 8, 'Nagpur': 9, 'Lucknow': 10}

{'Bhopal_1': 4,
 'Bhopal_2': 6,
 'Indore': 2,
 'Lucknow': 10,
 'Nagpur': 9,
 'Raipur': 8}

In [ ]:
find_franchises(8)

[21.260239, 81.6216235]

In [ ]:
int(_df.FranchiseId.mode().values[0])

NameError: ignored

# Distance Constraints

In [ ]:
def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            # Forbid transit from depot to depot node.
            ##################################################################
            elif from_node in range(data['depot_no']) and to_node in range(data['depot_no']):
                _distances[from_node][to_node] = data['vehicle_max_distance']
            ##################################################################
            else:
                _distances[from_node][to_node] = data['distance_matrix'][from_node][to_node]

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]
    return distance_evaluator

In [ ]:
def add_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span constraint"""
    distance = 'Distance'
    routing.AddDimension(
        distance_evaluator_index, #callback that returns the distance weightage.
        0,  # slack_max, slack represents waiting times at the location before leaving for next, here slack_max is Maximum Waiting Time allowed to a vehicle.
        data['vehicle_max_distance'],  # maximum distance per vehicle.
        True,  # start cumul to zero, if True Start cumulation of distance from Start Point, if false Start cumulation of distance after Node Point.
        distance) # Dimension Name
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    # global_span_cost = coefficient * (Max(cumul distance at end) - Min(cumul distance at start))

# Capacity Contraint

In [ ]:

def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator

In [ ]:
def add_capacity_constraints(routing, manager, data, demand_evaluator_index,node_to_remove):
    """Adds capacity constraint"""
    vehicle_capacity = data['vehicle_capacity']
    capacity = 'Capacity'
    routing.AddDimensionWithVehicleCapacity(
        demand_evaluator_index, #callback that returns the Capacity weightage.
        0,  # null capacity slack, slack is  maximum capacty to load before leaving for next visit.
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero, if True Start cumulation of distance from Start Point, if false Start cumulation of Capacity after Node Point.
        capacity) # Dimension Name
    
    # Add Slack for reseting to zero unload depot nodes.
    # e.g. vehicle with load 10/15 arrives at node 1 (depot unload)
    # so we have CumulVar = 10(current load) + -15(unload) + 5(slack) = 0.
    capacity_dimension = routing.GetDimensionOrDie(capacity)

    # Allow to drop reloading/unloading nodes with zero cost.
    for node in range(1,data['depot_no']): # Skipping Node 0 as it is our Depot Point.
      node_index = manager.NodeToIndex(node)
      routing.AddDisjunction([node_index], 0)  # Cost of Droping Extra Unload Points is zero.

    # Allow to drop regular node with a cost.
    # Required for Least Vehicle Approch
    penalty = 10**15   # Higher Penalty to avoid droping of nodes
    for node in range(data['depot_no'], data['num_locations']):
      if node in node_to_remove:
        print(f'node: {node} allocated Zero Penality')
        routing.AddDisjunction([manager.NodeToIndex(node)], 0)  # Zero Penalty to drop unwanted node while computing.
      else:
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)


In [ ]:
def create_time_evaluator(data):
    """Creates callback to get total times between locations."""

    def service_time(data, node):
        """Gets the service time for the specified location."""
        return abs(data['demands'][node]) * data['time_per_demand_unit']

    def travel_time(data, from_node, to_node):
        """Gets the travel times between two locations."""
        if from_node == to_node:
            travel_time = 0
        else:
            # travel_time = manhattan_distance(
            #         data['locations'][from_node], data['locations'][to_node]) / data['vehicle_speed']
            travel_time = data['time_matrix'][from_node][to_node]
        return travel_time

    _total_time = {}
    # precompute total time to have time callback in O(1)
    for from_node in range(data['num_locations']):
        _total_time[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _total_time[from_node][to_node] = 0
            else:
                _total_time[from_node][to_node] = int(
                    service_time(data, from_node) + travel_time(
                        data, from_node, to_node))

    def time_evaluator(manager, from_node, to_node):
        """Returns the total time between the two nodes"""
        return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return time_evaluator


In [ ]:
def add_time_window_constraints(routing, manager, data, time_evaluator):
    """Add Time windows constraint"""
    time = 'Time'
    max_time = data['vehicle_max_time']
    routing.AddDimension(
        time_evaluator,
        max_time,  # allow waiting time
        max_time,  # maximum time per vehicle
        False,  # don't force start cumul to zero since we are giving TW to start nodes
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
        routing.AddToAssignment(time_dimension.SlackVar(index))
    # Add time window constraints for each vehicle start node
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
        routing.AddToAssignment(time_dimension.SlackVar(index))
        # Warning: Slack var is not defined for vehicle's end node
        #routing.AddToAssignment(time_dimension.SlackVar(self.routing.End(vehicle_id)))



In [ ]:

###########
# Printer #
###########
def print_solution(data, manager, routing, assignment):  # pylint:disable=too-many-locals
    global routes
    global routes_data
    routes_data = []
    routes =[]
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    time_dimension = routing.GetDimensionOrDie('Time')
    dropped = []
    for order in range(data['depot_no'], routing.nodes()):
        index = manager.NodeToIndex(order)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(order)
    print(f'dropped orders: {dropped}')
    for reload in range(1,data['depot_no']):
        index = manager.NodeToIndex(reload)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(reload)
    print(f'dropped reload stations: {dropped}')

    for vehicle_id in range(data['num_vehicles']):

        # Temp list to store visit points in a route
        _temp_list =[]

        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            plan_output += ' {0} Load({1}) Time({2},{3}) ->'.format(
                manager.IndexToNode(index),
                assignment.Value(load_var),
                assignment.Min(time_var), assignment.Max(time_var))

            # Append visits in a route to list
            _temp_list.append(manager.IndexToNode(index))

            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
            
        # Normalizing Distance After Vehicle Cost is Added in Distance
        distance %=10**8

        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        plan_output += ' {0} Load({1}) Time({2},{3})\n'.format(
            manager.IndexToNode(index),
            assignment.Value(load_var),
            assignment.Min(time_var), assignment.Max(time_var))

        # Append End visits to list
        _temp_list.append(manager.IndexToNode(index))

        
        plan_output += f'Distance of the route: {distance}m\n'
        plan_output += f'Load of the route: {assignment.Value(load_var)}\n'
        plan_output += f'Time of the route: {assignment.Value(time_var)}min\n'
        print(plan_output)
     
        # Adding Data to routes_data
        routes_data.append({'distance':distance, 'time_taken':assignment.Value(time_var),'load':assignment.Value(load_var)})
        
        total_distance += distance
        total_load += assignment.Value(load_var)
        total_time += assignment.Value(time_var)
        
        # Append visits to routes list
        routes.append(_temp_list)
        #############################################

    print('Total Distance of all routes: {}m'.format(total_distance))
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Time of all routes: {}min'.format(total_time))



In [ ]:
# User Defined Fuction to add Weight and Quanlity per vehicle from pickup data.
def total_quantity(routes,rs_data):
  main_df = _df.fillna(0).reset_index()
  for i in range(len(routes)):
    temp_df = main_df.iloc[routes[i]]
    rs_data[i]['weight'] = temp_df.weight.sum()
    rs_data[i]['piece'] = temp_df.piece.sum()
    rs_data[i]['requestCompleted'] = len(routes[i])-2
  return rs_data

In [ ]:
# User Defined Function to Extract Nodes in a Same Vehicles from Routes.
# Output used as sameVehicleNodes in Vrp Fuction to Add or Remove visits without changing older routes.
def modify_route(a_list, node_to_remove):
  temp_list = a_list[:]
  temp_list = [[node for node in route if (node not in node_to_remove)&(node!=0)] for route in temp_list  ]
  # Removing Empty routes in from Current Route
  temp_list = [x for x in temp_list if not ((len(x)==0))]
  return temp_list

In [ ]:
def add_visit(df,visits_to_add):
  df=df[:]
  length_of_df = len(df)
  columns = list(df)
  for i in range(visits_to_add):
    placeholder = 1
    while placeholder is not None:
      try:
        RequestId = int(input('Enter ID:'))
        AddressTypeId = int(input('Enter AddressTypeId: ' ) or '1')
        FranchiseId = int(input('Enter FranchiseId: ' ) or '0')
        Latitude = float(input('Enter Latitude: '))
        Longitude = float(input('Enter Longitude: '))
        expected_weight = int(input('Enter Expected Weight in Kg: '))
        placeholder = None
      except Exception as e:
        placeholder = e
        print('error occurred try again!', e)
        continue
    
    values = [RequestId, 0	,AddressTypeId, "TEST",FranchiseId,	Latitude, Longitude,	0,	0,	expected_weight	,0]
    df.loc[length_of_df+i] = values
  return df

# **Main**

In [ ]:
def vrp(vehicle_no,df,depot_no,start=[],end = [],sameVehicleNodes =[],initialRoutes=[],node_to_remove=[]):

# Vehicle_no: Maximum Number of Vehicles available for rouuting.

# df: Dataframe with all requests data. Refer Importing File for file type or data in DataFrame

# depot_no: Number of Depots, Must be >=1.
#   if depot_no is 1: All request will be forced to unload at end of trip.
#   if deport_no is >1 : Enable unloading as soon as vehicle capacity is maxed out. 
#     this number indicated the number of total unloading points in middle of routes.

# start/end: List of Start/End points to force routes to start/end at a  given node rather than 'depot node' 0.
#   size of list must be equal to vehicle_no.

# sameVehicleNodes: Force given nodes to be in a same route. type: list of lists ex: [[route1_nodes],[route2_nodes],....,[route_n_nodes]]
#   size of list must be equal to vehicle_no.

# intialRoutes: Force the vrp to follow initial route in solution.  type: list of lists ex: [[route1_nodes],[route2_nodes],....,[route_n_nodes]]
#   size of list must be equal to vehicle_no.

# node_to_remove: list of nodes to be dropped in vrp.

############## Checking Valid Start or End Point  ############################
    if len(start)!=vehicle_no or len(end)!= vehicle_no:
        print('length of start list/ endlist does not match with number of vehicle')
        print('initializing start and end to 0')
        start = [0 for x in range(vehicle_no)]
        end = [0 for x in range(vehicle_no)]
      
    global routes_data
    global data
    data = open_data(vehicle_no,df,depot_no)
    data['starts'] =start[:]
    data['ends'] = end[:]
    print('Start_list:',data['starts'])
    print('Ends_list:',data['ends'])

    # The routing solver uses variable indices internally and these variable indices are tricky to manage. 
    # RoutingIndexManger simplify variable index usage, allowing users to use NodeIndex instead. Usage. Manager for any NodeIndex <-> variable index conversion.
    # RoutingIndexManager manager(10, 4, starts_ends); // 10 nodes, 4 vehicles,0(start and end node)
    # For Different Start and End node use: data['starts'] and data['ends'] instead of data['depot']
    # data['starts']/data['ends']  is list of Start/End index, Number of elements in list must be equal to number of vehicles.

    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'],  data['starts'],data["ends"])
    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    #Set Fixed Cost for all Vehicle:
    # Required for Least Vehicle Approach.
    routing.SetFixedCostOfAllVehicles(10**8)

    '''
    Vehicle Drop Cost is Less than Visit Drop cost.
    As we need to fulfill all our requests with least number of vehicles.
    '''

    # Define weight of each edge
    distance_evaluator_index = routing.RegisterTransitCallback(partial(create_distance_evaluator(data), manager)) #The index for the callback that returns the quantity
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index) # Set cost between nodes.

    # Add Distance constraint to minimize the longest route
    add_distance_dimension(routing, manager, data, distance_evaluator_index)

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index,node_to_remove) 

    # Add Time Window constraint
    time_evaluator_index = routing.RegisterTransitCallback(
        partial(create_time_evaluator(data), manager))
    add_time_window_constraints(routing, manager, data, time_evaluator_index)

    ################################################################################

    # Removing Nodes from Current Route
    sameVehicleNodes = modify_route(sameVehicleNodes,node_to_remove)
    penalty = 10**12
    if len(sameVehicleNodes) !=0:
      print('solving for Nodes in Same route',sameVehicleNodes)
      for i in range(len(sameVehicleNodes)):
          routing.AddSoftSameVehicleConstraint(sameVehicleNodes [i], penalty)

    if len(initialRoutes) !=0:
      initialRoutes = modify_route(initial_routes,node_to_remove)
    #   initial_solution = routing.ReadAssignmentFromRoutes(initialRoutes, True)

    ################################################################################


    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # pylint: disable=no-member
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(20)
    search_parameters.log_search = True

    # Solve the problem.
    if len(initialRoutes) != 0:
      routing.CloseModel()
      print(routing.ApplyLocksToAllVehicles(initialRoutes, False)) 
      print('solution with Intial Routes :',initialRoutes)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


    routes_data = total_quantity(routes,routes_data)
if __name__ == '__main__':
    vrp(5,_Pickup,1)

In [ ]:
if __name__ == '__main__':
    vrp(5,_Pickup,1)

# Add Visit

In [ ]:
_Pickup.shape

In [ ]:
initial_routes = routes[:]

In [ ]:
add_visit_df = add_visit(_Pickup,1)

In [ ]:
print(initial_routes)

In [ ]:
# adding initial_routes as sameVehicleNodes to avoid Change in Initial Routes
# New routes are stored in routes variable.
vrp(5,add_visit_df,1,
    start=[0,0,0,0,0],
    end = [0,0,0,0,0],
    sameVehicleNodes = initial_routes,
    initialRoutes= [ # Completed Visits
                    [9, 10, 11, 14, 19], 
                    [ 3, 4, 5, 7, 8, 6],
                    [],
                    []
    ])

In [ ]:
print(routes)

In [ ]:
print(initial_routes)

# Dropping Visit

In [ ]:
vrp(5,_Pickup,1,
    node_to_remove=[12],
    sameVehicleNodes=initial_routes,
    initialRoutes= [ # Completed Visits
                    [9, 10, 11, 14, 19], 
                    [ 3, 4, 5, 7, 8, 6],
                    [],
                    []
    ])
# adding initial_routes as sameVehicleNodes to avoid Change in Initial Routes
# New routes are stored in routes variable.

length of start list/ endlist does not match with number of vehicle
initializing start and end to 0
Start_list: [0, 0, 0, 0, 0]
Ends_list: [0, 0, 0, 0, 0]
node: 12 allocated Zero Penality
solving for Nodes in Same route [[9, 10, 11, 14, 19, 18, 17, 16, 13, 1], [3, 4, 5, 7, 8, 6, 15, 20, 2]]
True
solution with Intial Routes 
Objective: 203690515
dropped orders: []
dropped reload stations: []
Route for vehicle 0:
 0 Load(0) Time(0,0) -> 9 Load(0) Time(2,117) -> 10 Load(45) Time(34,149) -> 11 Load(62) Time(45,160) -> 14 Load(144) Time(91,206) -> 19 Load(149) Time(95,210) -> 18 Load(163) Time(103,218) -> 17 Load(172) Time(111,226) -> 16 Load(184) Time(117,232) -> 13 Load(194) Time(129,244) -> 12 Load(242) Time(160,275) -> 1 Load(313) Time(210,325) -> 0 Load(619) Time(365,480)
Distance of the route: 36133m
Load of the route: 619
Time of the route: 365min

Route for vehicle 1:
 0 Load(0) Time(0,0) -> 3 Load(0) Time(12,231) -> 4 Load(18) Time(30,249) -> 5 Load(62) Time(54,273) -> 7 Load(71) T

In [ ]:
routes

[[0, 9, 10, 11, 14, 19, 18, 17, 16, 13, 12, 1, 0],
 [0, 3, 4, 5, 7, 8, 6, 15, 20, 2, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

# User Defined Global Variables


*   pickup: DataFrame with Pickup data

*   _Pickup: Optimized and Clean Version of pickup dataframe
*   _df : _Pickup with defined number of Depot Points.
*   routes: Output of Routing and Path in List format 
*   original_routes: Manual Path calculated using VehicleSessionId and Time of *   completetion
*   route_data: Contains data for individual Routes.
*   original_routes_data: Contains data of individual manual routes.



### Extracting Original Routes

In [ ]:
unique_route_id = _Pickup.PickupSessionId.unique()
original_routes = []
for i in unique_route_id:
  temp_list = list(_Pickup[_Pickup.PickupSessionId == i].index.values)
  temp_list = [ x+ int(data['depot_no']) for x in temp_list]
  original_routes.append([0]+temp_list+[0])

## CALCULATE DISTANCE TRAVELLED IN ORIGINAL ROUTES


In [ ]:
original_routes_data =[{} for _ in original_routes ]
main_df = _df.fillna(0).reset_index()

for i in range (len(original_routes)):
  previous_visit = 0
  distance = 0
  time_taken = 0
  load = 0
  for visit in original_routes[i]:
    time_as_per_demand = data['demands'][visit] * data['time_per_demand_unit']
    load+=data['demands'][visit]
    distance += data['distance_matrix'][previous_visit][visit]
    time_taken += data['time_matrix'][previous_visit][visit]+time_as_per_demand
    previous_visit = visit
  original_routes_data[i]['distance'] = distance
  original_routes_data[i]['time'] = time_taken
  original_routes_data[i]['load'] = load
  temp_df = main_df.iloc[original_routes[i]]
  original_routes_data[i]['weight'] = int(temp_df.weight.sum())
  original_routes_data[i]['piece'] = int(temp_df.piece.sum())
  original_routes_data[i]['requestCompleted'] = len(original_routes[i])-2

In [ ]:
# Manual Routes 
original_routes

[[0, 1, 0],
 [0, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0]]

In [ ]:
# VRP Routes
print(routes)

[[0, 9, 6, 8, 7, 5, 4, 3, 0], [0, 0], [0, 0], [0, 0], [0, 2, 20, 15, 16, 17, 18, 19, 14, 13, 12, 11, 10, 1, 0]]


In [ ]:
temp_df = pd.DataFrame(original_routes_data)
temp_df.loc['Column Total']= temp_df.sum(numeric_only=True, axis=0)
display(HTML('<h1>Actual Route Data</h1>'))
display(HTML(temp_df.to_html()))

temp_df = pd.DataFrame(routes_data)
temp_df.loc['Column Total']= temp_df.sum(numeric_only=True, axis=0)
display(HTML('<h1>Predicted Route Data</h1>'))
display(HTML(temp_df.to_html()))

,distance,time,load,weight,piece,requestCompleted
0,1141.0,157.0,306.0,306.0,315.0,1.0
1,39318.2,275.5,417.0,422.0,6.0,8.0
2,38456.9,204.0,284.0,289.0,0.0,11.0
Column Total,78916.1,636.5,1007.0,1017.0,321.0,20.0


,distance,time_taken,load,weight,piece,requestCompleted
0,34648.0,234.0,357.0,361.12,2.0,7.0
1,0.0,0.0,0.0,0.00,0.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0
4,35711.0,387.0,650.0,656.74,319.0,13.0
Column Total,70359.0,621.0,1007.0,1017.86,321.0,20.0


# **MAP**

In [ ]:
import gmplot
import folium
import polyline

In [ ]:
predefined_colors = ['#1890FF','#52c41a','#faad14','#ff4d4f','#ffffff','#ff7f27']

In [ ]:
def get_route(pickup_lat,pickup_lon, dropoff_lat , dropoff_lon):
    global res
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes_x = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    routes_df = pd.DataFrame.from_records(routes_x, columns =['Latitude', 'Longitude'])
    #print(r.status_code)
    return routes_df

In [ ]:
def plot_routes(routes,data):
    Sample = data[:]
    gmap3 = gmplot.GoogleMapPlotter(23.1993477,77.2658056, 13)
    gmap3.coloricon = "http://www.googlemapsmarkers.com/v1/%s/"
    #ploting updated routes
    # routes = Original_Routes
    # scatter method of map object
    # scatter points on the google map
    for i in range(len(routes)):
        latitude_list = Sample.iloc[routes[i]]['Latitude'].values
        longitude_list = Sample.iloc[routes[i]]['Longitude'].values
        center_lat = latitude_list[0]
        center_long = longitude_list[0]
        #gmap3.scatter(center_lat , center_long, '# 00000',size = 100, marker = False )
        gmap3.scatter( latitude_list, longitude_list, color ='black',size = 10, marker = False)
        for j in range(len(latitude_list)):
            start_lat = latitude_list[j]
            start_long = longitude_list[j]
            #print(f'start_lat:{start_lat}, start_long{start_long}')
            try:
                stop_lat = latitude_list[j+1]
                stop_long = longitude_list[j+1]
                #print(f'stop_lat:{stop_lat}, stop_long{stop_long}')
            except:
                stop_lat = latitude_list[0]
                stop_long = longitude_list[0]
                #print('try triggered!')
                #print(f'stop_lat:{stop_lat}, stop_long{stop_long}')

            route_df = get_route(start_lat,start_long, stop_lat , stop_long)
            #print(len(route_df))
            route_latitude_list = route_df['Latitude'].values
            route_longitude_list = route_df['Longitude'].values
            #print(len(route_longitude_list))
    #         gmap3.scatter( route_latitude_list, route_longitude_list, '# FF0000',size = 0, marker = False )
            gmap3.plot(route_latitude_list, route_longitude_list,color=predefined_colors[i] , edge_width = 2.5)

    gmap3.marker(23.263742, 77.454773,color = "green", title="Depot")
    Sample_2 = Sample[~(Sample.AddressTypeId ==0)][['Latitude','Longitude']]
    for index, row in Sample_2.iterrows():
      lat = row.Latitude
      lng = row.Longitude
      gmap3.marker(lat, lng, title=str(index),color='cornflowerblue')

      

    gmap3.draw( r"/content/route.html" )
    print('Route Ploted on route.html')

In [ ]:
plot_routes(original_routes,_df)

Route Ploted on route_1.html
